https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/

In [58]:
import pickle
import albumentations as A
from torchvision import transforms, datasets
import torch.utils.data as data
from PIL import Image
import os
import json
from torchvision import transforms
import random
import numpy as np
import json
import torch
import pickle
import cv2
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler


import gc
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt
import timm
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer

import math


# DATASET

In [2]:
from tqdm import tqdm
path_to_image = {}
for i in tqdm(range(0, 23)):
    path = '/scratch/sm9669/nabird pickle/nabird/train_'+str(i*1000)+'.pkl'
    file = open(path,'rb')
    sub_data = pickle.load(file)
    path_to_image.update(sub_data)

100%|██████████| 23/23 [00:44<00:00,  1.92s/it]


In [3]:
with open('/scratch/sm9669/classes.txt') as f:
    lines = f.readlines()
classid_name = []
for x in lines:
    classid_name.append( x[len(x.split(" ")[0])+1:-1])

In [4]:
classid_name

['Birds',
 'Ducks, Geese, and Swans',
 'Grouse, Quail, and Allies',
 'Loons',
 'Grebes',
 'Storks',
 'Frigatebirds, Boobies, Cormorants, Darters, and Allies',
 'Pelicans, Herons, Ibises, and Allies',
 'Hawks, Kites, Eagles, and Allies',
 'Caracaras and Falcons',
 'Cranes and Rails',
 'Plovers, Sandpipers, and Allies',
 'Skuas and Alcids',
 'Gulls, Terns, and Allies',
 'Pigeons and Doves',
 'Parrots',
 'Cuckoos',
 'Owls',
 'Nightjars',
 'Swifts and Hummingbirds',
 'Kingfishers and Allies',
 'Woodpeckers',
 'Perching Birds',
 'Black-bellied Plover',
 'Black-bellied Whistling-Duck',
 'Tyrant Flycatchers',
 'Semipalmated Plover',
 'Greater White-fronted Goose',
 'Black Vulture',
 'American White Pelican',
 'Killdeer',
 'Red-headed Woodpecker',
 'Shrikes',
 'Snow Goose',
 'Chimney Swift',
 'American Oystercatcher',
 "Ross's Goose",
 'Barn Owl',
 'Vireos',
 'Black Oystercatcher',
 'Turkey Vulture',
 'Brown Pelican',
 'Brant',
 'Scaled Quail',
 'Northern Gannet',
 'Rock Pigeon',
 'Black-necke

In [5]:
def imgpath_name(path):
    class_id = int(path.split('/')[0])
    return classid_name[class_id]

In [6]:
meta_data = '/scratch/sm9669/nabirds_with_loc_2019.json'

In [77]:
class CFG:
    debug = False
#     image_path = image_path
#     captions_path = captions_path
    batch_size = 32
    num_workers = 2
    head_lr = 1e-3
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    meta_encoder_lr = 0.0005
    weight_decay = 1e-3
    patience = 1
    factor = 0.8
    epochs = 3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_name = 'resnet50'
    image_embedding = 2048
    meta_embedding = 256
    text_encoder_model = "distilbert-base-uncased"
    text_embedding = 768
    text_tokenizer = "distilbert-base-uncased"
    max_length = 200

    pretrained = True # for both image encoder and text encoder
    trainable = True # for both image encoder and text encoder
    temperature = 1.0

    # image size
    size = 224

    # for projection head; used for both image and text encoders
    num_projection_layers = 1
    projection_dim = 256 
    dropout = 0.1

In [8]:
params={}
params['loc_encode'] = 'encode_cos_sin'
params['use_date_feats']=True
params['date_encode'] = 'encode_cos_sin'
params['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

In [23]:
import numpy as np
import torch
import json
import os
import math
from torch.utils.data.sampler import Sampler


def encode_loc_time(loc_ip, date_ip, concat_dim=1, params=None):
    # assumes inputs location and date features are in range -1 to 1
    # location is lon, lat

    if params['loc_encode'] == 'encode_cos_sin':
        feats = torch.cat((torch.sin(math.pi*loc_ip), torch.cos(math.pi*loc_ip)), concat_dim)

    elif params['loc_encode'] == 'encode_3D':
        # X, Y, Z in 3D space
        if concat_dim == 1:
            cos_lon = torch.cos(math.pi*loc_ip[:, 0]).unsqueeze(-1)
            sin_lon = torch.sin(math.pi*loc_ip[:, 0]).unsqueeze(-1)
            cos_lat = torch.cos(math.pi*loc_ip[:, 1]).unsqueeze(-1)
            sin_lat = torch.sin(math.pi*loc_ip[:, 1]).unsqueeze(-1)
        if concat_dim == 2:
            cos_lon = torch.cos(math.pi*loc_ip[:, :, 0]).unsqueeze(-1)
            sin_lon = torch.sin(math.pi*loc_ip[:, :, 0]).unsqueeze(-1)
            cos_lat = torch.cos(math.pi*loc_ip[:, :, 1]).unsqueeze(-1)
            sin_lat = torch.sin(math.pi*loc_ip[:, :, 1]).unsqueeze(-1)
        feats = torch.cat((cos_lon*cos_lat, sin_lon*cos_lat, sin_lat), concat_dim)

    elif params['loc_encode'] == 'encode_none':
        feats = loc_ip

    else:
        print('error - no loc feat type defined')


    if params['use_date_feats']:
        if params['date_encode'] == 'encode_cos_sin':
            feats_date = torch.cat((torch.sin(math.pi*date_ip.unsqueeze(-1)),
                                    torch.cos(math.pi*date_ip.unsqueeze(-1))), concat_dim)
        elif params['date_encode'] == 'encode_none':
            feats_date = date_ip.unsqueeze(-1)
        else:
            print('error - no date feat type defined')
        feats = torch.cat((feats, feats_date), concat_dim)

    return feats



def convert_loc_to_tensor(x, device=None):
    # intput is in lon {-180, 180}, lat {90, -90}
    xt = x.astype(np.float32)
    xt[:,0] /= 180.0
    xt[:,1] /= 90.0
    xt = torch.from_numpy(xt)
    if device is not None:
        xt = xt.to(device)
    return xt

In [24]:
def generate_feats(locs, dates):
    x_locs = convert_loc_to_tensor(locs)
    x_dates = torch.from_numpy(dates.astype(np.float32)*2 - 1)
    feats = encode_loc_time(x_locs, x_dates, concat_dim=1, params=params)
    return feats

In [25]:

class CLIPDataset(torch.utils.data.Dataset):
    
    def default_loader(self, path):
        image= path_to_image[path]
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        im_pil = Image.fromarray(img)
    #     return Image.open(path).convert('RGB')
        return im_pil
    
    def __init__(self, meta_data, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """
        meta_type = 'ebird_meta'
        
        print('Loading annotations from: ' + os.path.basename(meta_data))
        with open(meta_data) as fp:
            da = json.load(fp)

        # only choose images that are available - ie some missing from download
        # also select the split of interest
        da = [dd for dd in da['train'] if dd['valid_image'] and not math.isnan(dd['ebird_meta']['lat'])]

        # set up the filenames and annotations
        self.imgs = [dd['im_path'] for dd in da]
        self.classes = [dd['class_id'] for dd in da]
        self.users = [dd[meta_type]['user_id'] for dd in da]
        self.dates = [dd[meta_type]['date'] for dd in da]
        self.lon = [dd[meta_type]['lon'] for dd in da]
        self.lat = [dd[meta_type]['lat'] for dd in da]
        self.name = [imgpath_name(dd['im_path']) for dd in da]
        
        
        lat = np.array(self.lat).reshape(-1,1)
        lon = np.array(self.lon).reshape(-1,1)
        dates = np.array(self.dates)
        location = np.concatenate([lat,lon],axis=1)
        self.feats = generate_feats(location,dates)
        
        
        self.captions = ["This is a photograph of "+ x for x in self.name]
        self.encoded_captions = tokenizer(
            list(self.captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        image = path_to_image[self.imgs[idx]]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['caption'] = self.captions[idx]
        item['lon'] = self.lon[idx]
        item['lat'] = self.lat[idx]
        item['dates'] = self.dates[idx]
        item['lat_lon_date_feats'] = self.feats[idx]
        item['class_id'] = self.classes[idx]

        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )

In [26]:
tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
dataset= CLIPDataset(meta_data, tokenizer,get_transforms())


validation_split = .2
shuffle_dataset = True
random_seed= 42


dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers,
        sampler=train_sampler
    )
valid_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers,
        sampler=valid_sampler
    )

Loading annotations from: nabirds_with_loc_2019.json


# MODEL

In [27]:
class ImageEncoder(nn.Module):
    """
    Encode images to a fixed size vector
    """

    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name, pretrained, num_classes=0, global_pool="avg"
        )
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [28]:
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name)
        else:
            self.model = DistilBertModel(config=DistilBertConfig())
            
        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

In [29]:
class ResLayer(nn.Module):
    def __init__(self, linear_size):
        super(ResLayer, self).__init__()
        self.l_size = linear_size
        self.nonlin1 = nn.ReLU(inplace=True)
        self.nonlin2 = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout()
        self.w1 = nn.Linear(self.l_size, self.l_size)
        self.w2 = nn.Linear(self.l_size, self.l_size)

    def forward(self, x):
        y = self.w1(x)
        y = self.nonlin1(y)
        y = self.dropout1(y)
        y = self.w2(y)
        y = self.nonlin2(y)
        out = x + y

        return out

In [30]:
class MetaEncoder(nn.Module):
    def __init__(self, num_inputs=6, num_classes=555, num_filts=256, num_users=1):
        super(MetaEncoder, self).__init__()
        self.inc_bias = False
        self.class_emb = nn.Linear(num_filts, num_classes, bias=self.inc_bias)

        self.feats = nn.Sequential(nn.Linear(num_inputs, num_filts),
                                    nn.ReLU(inplace=True),
                                    ResLayer(num_filts),
                                    ResLayer(num_filts),
                                    ResLayer(num_filts),
                                    ResLayer(num_filts))

    def forward(self, x, class_of_interest=None, return_feats=True):
        loc_emb = self.feats(x)
        if return_feats:
            return loc_emb
        if class_of_interest is None:
            class_pred = self.class_emb(loc_emb)
        else:
            class_pred = self.eval_single_class(loc_emb, class_of_interest)

        return torch.sigmoid(class_pred)

    def eval_single_class(self, x, class_of_interest):
        if self.inc_bias:
            return torch.matmul(x, self.class_emb.weight[class_of_interest, :]) + self.class_emb.bias[class_of_interest]
        else:
            return torch.matmul(x, self.class_emb.weight[class_of_interest, :])


In [31]:
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim,
        dropout=CFG.dropout
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [32]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
        meta_embedding=CFG.meta_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.meta_encoder= MetaEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.meta_projection = ProjectionHead(embedding_dim=meta_embedding)
        self.temperature = temperature

    def forward(self, batch):
        # Getting Image and Text Features
        dic={}
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        meta_features = self.meta_encoder(batch['lat_lon_date_feats'])
        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        meta_embeddings = self.meta_projection(meta_features)
        
        # Calculating the Loss
        def contrastive_loss(embedding_1, embedding_2):
            logits = (embedding_1 @ embedding_2.T) / self.temperature
            images_similarity = embedding_2 @ embedding_2.T
            texts_similarity = embedding_1 @ embedding_1.T
            targets = F.softmax(
                (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
            )
            texts_loss = cross_entropy(logits, targets, reduction='none')
            images_loss = cross_entropy(logits.T, targets.T, reduction='none')
            loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
            
            return loss
        
        loss_text_image = contrastive_loss(text_embeddings, image_embeddings)
        loss_meta_image = contrastive_loss(meta_embeddings, image_embeddings)
        loss_text_meta = contrastive_loss(text_embeddings, meta_embeddings)
    
        loss = loss_text_image +loss_meta_image +loss_text_meta
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [19]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]


# TRAIN

In [66]:
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter


In [71]:
def valid_epoch(model, valid_loader):
    loss_meter = AvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)
        count = batch["image"].to(CFG.device).size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(valid_loss=loss_meter.avg)
    return loss_meter


In [30]:
train_loss_list=[]
val_loss_list=[]

In [ ]:
model = CLIPModel().to(CFG.device)
params = [
    {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
    {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
    {"params": model.meta_encoder.parameters(), "lr": CFG.meta_encoder_lr},
    {"params": itertools.chain(
        model.image_projection.parameters(), model.text_projection.parameters() , model.meta_projection.parameters()
    ), "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
]
optimizer = torch.optim.AdamW(params, weight_decay=0.)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", patience=CFG.patience, factor=CFG.factor
)
step = "epoch"

best_loss = float('inf')
for epoch in range(CFG.epochs):
    print(f"Epoch: {epoch + 1}")
    model.train()
    train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
    train_loss_list.append(train_loss.avg)
    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(model, valid_loader)
        val_loss_list.append(valid_loss.avg)
    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        torch.save(model.state_dict(), "best.pt")
        print("Saved Best Model!")

    lr_scheduler.step(valid_loss.avg)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1


100%|██████████| 142/142 [00:19<00:00,  7.33it/s, valid_loss=10.3]


Saved Best Model!
Epoch: 2


100%|██████████| 142/142 [00:19<00:00,  7.46it/s, valid_loss=8.65]


...

100%|██████████| 142/142 [00:19<00:00,  7.25it/s, valid_loss=12.7]


Epoch: 79


100%|██████████| 142/142 [00:19<00:00,  7.16it/s, valid_loss=12.7]


Epoch: 80


 93%|█████████▎| 526/565 [01:32<00:06,  6.03it/s, lr=4.06e-8, train_loss=2.69]

In [ ]:
print(train_loss_list , val_loss_list)

In [ ]:
epoch_list=[i for i in range(len(train_loss_list))]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epoch_list, train_loss_list)
plt.plot(epoch_list, val_loss_list)
plt.show()

# Finetuned Model Training Code

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [59]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
        meta_embedding=CFG.meta_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.meta_encoder= MetaEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.meta_projection = ProjectionHead(embedding_dim=meta_embedding)
        self.temperature = temperature

    def forward(self, batch):
        # Getting Image and Text Features
        dic={}
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        meta_features = self.meta_encoder(batch['lat_lon_date_feats'])
        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        meta_embeddings = self.meta_projection(meta_features)
        
        # Calculating the Loss
        def contrastive_loss(embedding_1, embedding_2):
            logits = (embedding_1 @ embedding_2.T) / self.temperature
            images_similarity = embedding_2 @ embedding_2.T
            texts_similarity = embedding_1 @ embedding_1.T
            targets = F.softmax(
                (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
            )
            texts_loss = cross_entropy(logits, targets, reduction='none')
            images_loss = cross_entropy(logits.T, targets.T, reduction='none')
            loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
            
            return loss
        
        loss_text_image = contrastive_loss(text_embeddings, image_embeddings)
        loss_meta_image = contrastive_loss(meta_embeddings, image_embeddings)
        loss_text_meta = contrastive_loss(text_embeddings, meta_embeddings)
    
        loss = loss_text_image +loss_meta_image +loss_text_meta
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [60]:
class ClassifierHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        num_classes =555
    ):
        super().__init__()
        self.fc1 = nn.Linear(embedding_dim, embedding_dim)
        self.gelu = nn.GELU()
        self.fc2 = nn.Linear(embedding_dim, num_classes)
        self.dropout = nn.Dropout(0.5)
        
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
class ModelInfer(nn.Module):
    def __init__(
        self,
        PRETRAINED_MODEL = None
    ):
        super().__init__()
        self.image_encoder = PRETRAINED_MODEL.image_encoder
        self.meta_encoder= PRETRAINED_MODEL.meta_encoder
        self.image_projection = PRETRAINED_MODEL.image_projection
        self.meta_projection = PRETRAINED_MODEL.meta_projection
        self.classification_layer = ClassifierHead(512)
        
    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        meta_features = self.meta_encoder(batch['lat_lon_date_feats'])
        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        meta_embeddings = self.meta_projection(meta_features)
        
        out = torch.cat([image_embeddings,meta_embeddings], axis  = 1)
        
        predicted = self.classification_layer(out)
    
        loss = F.cross_entropy(predicted, batch['class_id'])
        return loss.mean()




In [61]:
model = CLIPModel()
model.load_state_dict(torch.load("best.pt"))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [62]:
infer = ModelInfer(model)

In [63]:
it = iter(train_loader)
first = next(it)
print(len(train_loader))

565


In [64]:
infer(first)

tensor(6.3162, grad_fn=<MeanBackward0>)

In [57]:
infer

ModelInfer(
  (image_encoder): ImageEncoder(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): ReLU(inplace=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05

In [78]:
CLIP = CLIPModel().to(CFG.device)
CLIP.load_state_dict(torch.load("best.pt"))
model = ModelInfer(CLIP).to(CFG.device)
params = [
    {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
    {"params": model.meta_encoder.parameters(), "lr": CFG.meta_encoder_lr},
    {"params": itertools.chain(
        model.image_projection.parameters(), model.classification_layer.parameters() , model.meta_projection.parameters()
    ), "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
]
optimizer = torch.optim.AdamW(params, weight_decay=0.)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", patience=CFG.patience, factor=CFG.factor
)
step = "epoch"

best_loss = float('inf')
train_loss_list=[]
val_loss_list=[]
for epoch in range(CFG.epochs):
    print(f"Epoch: {epoch + 1}")
    model.train()
    train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
    train_loss_list.append(train_loss.avg)
    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(model, valid_loader)
        val_loss_list.append(valid_loss.avg)
    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        torch.save(model.state_dict(), "best_inference.pt")
        print("Saved Best Model!")

    lr_scheduler.step(valid_loss.avg)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1


  0%|          | 0/565 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

Saved Best Model!
Epoch: 2


...

  0%|          | 0/142 [00:00<?, ?it/s]

Saved Best Model!
Epoch: 3


  0%|          | 0/565 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

Saved Best Model!
